[RAG](https://dspy.ai/tutorials/rag/) Using a DSPy Optimizer to improve your RAG prompt

In [1]:
import ujson
from dspy.utils import download

download("https://huggingface.co/dspy/cache/resolve/main/ragqa_arena_tech_corpus.jsonl")

/home/mike/git/dspy-tool-use/venv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import dspy
import ujson
from sentence_transformers import SentenceTransformer

max_characters = 6000  # for truncating >99th percentile of documents
topk_docs_to_retrieve = 5  # number of documents to retrieve per search query

with open("ragqa_arena_tech_corpus.jsonl") as f:
    corpus = [ujson.loads(line)['text'][:max_characters] for line in f]
    print(f"Loaded {len(corpus)} documents. Will encode them below.")

# same embedding model we use in our RAG
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2", device="cuda")
embedder = dspy.Embedder(model.encode)
search = dspy.retrievers.Embeddings(embedder=embedder, corpus=corpus, k=topk_docs_to_retrieve, brute_force_threshold=30000)

/home/mike/git/dspy-tool-use/venv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded 28436 documents. Will encode them below.


Batches: 100%|██████████| 1/1 [00:00<00:00, 24.60it/s]


In [4]:
import dspy
import openai
import os

LLM_URL=os.getenv('LLM_URL', 'http://localhost:8080/v1')
API_KEY=os.getenv('API_KEY', 'fake')
LLM_MODEL=os.getenv('LLM_MODEL', 'openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf')
MAX_TOKENS=os.getenv('MAX_TOKENS', 6000)
TEMPERATURE=os.getenv('TEMPERATURE', 0.2)
dspy.enable_logging()
lm = dspy.LM(model=LLM_MODEL,
             api_base=LLM_URL,  # ensure this points to your port
             api_key=API_KEY,
             temperature=TEMPERATURE,
             model_type='chat',
             stream=False)
dspy.configure(lm=lm)
#dspy.settings.configure(track_usage=True)

In [5]:
class RAG(dspy.Module):
    def __init__(self):
        self.respond = dspy.ChainOfThought('context, question -> response')

    def forward(self, question):
        context = search(question).passages
        return self.respond(context=context, question=question)

In [ ]:
rag = RAG()
rag(question="what are high memory and low memory on linux?")

Prediction(
    reasoning='High Memory and Low Memory are terms used to describe the division of memory space in a Linux system. High Memory refers to the segment of memory that user-space programs can access, while Low Memory is the segment that the Linux kernel can access directly. This division is necessary to prevent user-space applications from accessing kernel-space memory, which could potentially lead to security vulnerabilities.',
    response='High Memory and Low Memory are terms used to describe the division of memory space in a Linux system. High Memory refers to the segment of memory that user-space programs can access, while Low Memory is the segment that the Linux kernel can access directly. This division is necessary to prevent user-space applications from accessing kernel-space memory, which could potentially lead to security vulnerabilities. The Linux kernel splits the available memory into two parts: High Memory (user-space) and Low Memory (kernel-space), with the lat

12:45:23 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
12:45:23 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: models/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: models/Llama-3.2-3B-Instruct-Q8_0.gguf
12:45:23 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
12:45:23 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: models/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: models/Llama-3.2-3B-Instruct-Q8_0.gguf
12:45:31 - LiteLLM:INFO: cost_calculator.py:655 - selected model name fo

In [7]:
dspy.inspect_history()





[2025-06-03T12:22:46.432884]

System message:

Your input fields are:
1. `context` (str)
2. `question` (str)
Your output fields are:
1. `reasoning` (str)
2. `response` (str)
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## context ## ]]
{context}

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## response ## ]]
{response}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `context`, `question`, produce the fields `response`.


User message:

[[ ## context ## ]]
[1] «As far as I remember, High Memory is used for application space and Low Memory for the kernel. Advantage is that (user-space) applications cant access kernel-space memory.»
[2] «This is relevant to the Linux kernel; Im not sure how any Unix kernel handles this. The High Memory is the segment of memory that user-space programs can address. It cannot touch Low Memory. Low Memory is the segment of 

In [15]:
import random
import ujson

with open("ragqa_arena_tech_examples.jsonl") as f:
    data = [ujson.loads(line) for line in f]

data = [dspy.Example(**d).with_inputs('question') for d in data]

random.Random(0).shuffle(data)
# 200, 200:500, 500:1000
trainset, devset, testset = data[:50], data[50:150], data[150:450]

len(trainset), len(devset), len(testset)

(50, 100, 300)

In [16]:
from dspy.evaluate import SemanticF1

# Instantiate the metric.
metric = SemanticF1(decompositional=True)

# Define an evaluator that we can re-use.
evaluate = dspy.Evaluate(devset=devset, metric=metric, num_threads=24,
                         display_progress=True, display_table=2)

evaluate(RAG())

  0%|          | 0/100 [00:00<?, ?it/s]

Average Metric: 55.82 / 100 (55.8%): 100%|██████████| 100/100 [12:18<00:00,  7.39s/it]

2025/06/03 12:40:10 INFO dspy.evaluate.evaluate: Average Metric: 55.81709025389931 / 100 (55.8%)


,question,example_response,gold_doc_ids,reasoning,pred_response,SemanticF1
0,does using == in javascript ever make sense?,"Yes, using `==` in JavaScript can make sense and is convenient in ...","[5778, 5791, 5818]",The use of `==` in JavaScript can be misleading due to its behavio...,"Yes, using `==` in JavaScript can make sense in certain situations...",✔️ [0.667]
1,what is the difference between a virus and trojan?,The terms have a great deal of overlap and aren't necessarily mutu...,"[3768, 3769, 3888, 3890, 4046]",The difference between a virus and a Trojan lies in how they sprea...,"A virus and a Trojan are both types of malware, but they differ in...",✔️ [0.600]


55.82

In [17]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5500")
mlflow.set_experiment("optimize-rag")
mlflow.dspy.autolog(
    log_compiles=True,    # Track optimization process
    log_evals=True,       # Track evaluation results
    log_traces_from_compile=True  # Track program traces during optimization
)

2025/06/03 12:44:43 INFO mlflow.tracking.fluent: Experiment with name 'optimize-rag' does not exist. Creating a new experiment.


In [18]:
tp = dspy.MIPROv2(metric=metric, auto="medium", num_threads=24)  # use fewer threads if your rate limit is small

optimized_rag = tp.compile(RAG(), trainset=trainset,
                           max_bootstrapped_demos=2, max_labeled_demos=2,
                           requires_permission_to_run=False)

2025/06/03 12:45:14 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'a3b3616e10834427b5ac20b2cd63de46', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current dspy workflow
2025/06/03 12:45:14 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 18
minibatch: False
num_fewshot_candidates: 12
num_instruct_candidates: 6
valset size: 40

2025/06/03 12:45:14 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/06/03 12:45:14 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/06/03 12:45:14 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=12 sets of demonstrations...


Bootstrapping set 1/12
Bootstrapping set 2/12
Bootstrapping set 3/12


  0%|          | 0/10 [00:00<?, ?it/s]12:45:15 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= models/Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= models/Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/chat/completions "HTTP/1.1 200 OK"
12:45:23 - LiteLLM:INFO: utils.py:1213 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
12:45:23 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
12:45:23 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
12:45:24 - Lit

Bootstrapped 2 full traces after 8 examples for up to 1 rounds, amounting to 8 attempts.


Bootstrapping set 4/12


...

Average Metric: 26.20 / 40 (65.5%): 100%|██████████| 40/40 [00:06<00:00,  5.83it/s]

2025/06/03 14:27:22 INFO dspy.evaluate.evaluate: Average Metric: 26.198653986469825 / 40 (65.5%)
2025/06/03 14:27:22 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 65.5 with parameters ['Predictor 0: Instruction 5', 'Predictor 0: Few-Shot Set 4'].
2025/06/03 14:27:22 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [57.21, 60.52, 61.11, 60.85, 61.11, 55.55, 60.94, 62.5, 62.12, 59.76, 63.27, 63.27, 61.13, 61.3, 59.99, 60.68, 65.77, 65.77, 65.5]
2025/06/03 14:27:22 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 65.77
2025/06/03 14:27:22 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/06/03 14:27:22 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 65.77!



🏃 View run eval_full_18 at: http://localhost:5500/#/experiments/344816129373506955/runs/c6d520b1aeb2449fbc760565d9735561
🧪 View experiment at: http://localhost:5500/#/experiments/344816129373506955


🏃 View run resilient-shrike-234 at: http://localhost:5500/#/experiments/344816129373506955/runs/a3b3616e10834427b5ac20b2cd63de46
🧪 View experiment at: http://localhost:5500/#/experiments/344816129373506955


[Trace(request_id=315875f801ef44308cd7e744ffeadab8), Trace(request_id=c65eaa3b814641869caf4291aa52ea37), Trace(request_id=8796a4b478664e2b867cc3f3e06adb17), Trace(request_id=c2855dc0b1d44b2d85a5c3e483a571be), Trace(request_id=5b4de4a3edd24b5682fc02a189f36d2e), Trace(request_id=cc3dccbfcf8f45d1b467d0119aa256a0), Trace(request_id=dd8ecbcd036041188b4902f9493a5f85), Trace(request_id=ccecdfbb93f24595acf76060ebfc5363), Trace(request_id=7bc5e8cafdd14ae8a2691f5de035645b), Trace(request_id=b797735afa1e49e2a5b55a651e4b8699)]

In [19]:
baseline = rag(question="cmd+tab does not work on hidden or minimized windows")
print(baseline.response)

15:19:43 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= models/Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= models/Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/chat/completions "HTTP/1.1 200 OK"
15:19:50 - LiteLLM:INFO: utils.py:1213 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
15:19:50 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
15:19:50 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
15:19:50 - LiteLLM:INFO: cost_calculator.py:655 - se

You can use the following methods to switch between hidden or minimized windows on a Mac:
- Use `Control-CMD-F` to switch to full screen mode, which will show all open windows.
- Navigate to System Preferences > Mission Control, and uncheck "When switching to an application, switch to a Space with open windows for the application."
- Alternatively, you can use Automator Services or third-party applications like iCanHazShortcut to execute an `osascript` command that can handle hidden or minimized windows.
- Another approach is to use `Cmd+` (backtick) and press `Cmd+tab` while holding `Cmd`, which will show all open applications with animated displayed windows, allowing you to navigate using arrow keys.


Trace(request_id=3094bf7f5dda413aad0e91acf87c63f0)

In [20]:
pred = optimized_rag(question="cmd+tab does not work on hidden or minimized windows")
print(pred.response)

15:19:52 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= models/Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= models/Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/chat/completions "HTTP/1.1 200 OK"
15:20:02 - LiteLLM:INFO: utils.py:1213 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
15:20:02 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
15:20:02 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf


To access minimized windows using cmd+tab, try the following workarounds:
- Use Control-CMD-F for full screen mode to access minimized windows.
- Switch to all open apps before hiding.
- Use the following AppleScript to switch to the first minimized window: `delay 0.5 set i to 0 tell application System Events set first_app to name of the first process whose frontmost is true repeat with p in every process if visible of p then set i to i + 1 end if end repeat repeat i - 1 times key down command key down shift keystroke tab delay 0.01 key up shift key up command delay 0.1 end repeat set visible of process first_app to false end tell
- Alternatively, use the following shortcut in iCanHazShortcut: `osascript <scriptname>`
- Navigate to the minimized Application by doing Command+Tab while still holding Command, then release both keys.
- Press cmd+` to access minimized windows, but first press cmd+tab to open the window, then press cmd+` to minimize it.


Trace(request_id=2c3b532ff5f549d9bc88381569b506c7)

In [ ]:
evaluate(optimized_rag)

  0%|          | 0/100 [00:00<?, ?it/s]

15:20:13 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= models/Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= models/Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
15:20:13 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= models/Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= models/Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
15:20:16 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= models/Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= models/Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
15:20:16 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= models/Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= models/Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
15:20:16 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= models/Llama-3.2-3

Average Metric: 0.85 / 1 (84.7%):   1%|          | 1/100 [00:48<1:20:04, 48.53s/it]

15:21:01 - LiteLLM:INFO: utils.py:2991 - 
LiteLLM completion() model= models/Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= models/Llama-3.2-3B-Instruct-Q8_0.gguf; provider = openai
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/chat/completions "HTTP/1.1 200 OK"
15:21:06 - LiteLLM:INFO: utils.py:1213 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
15:21:06 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
15:21:06 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
15:21:06 - LiteLLM:INFO: cost_calculator.py:655 - se

Average Metric: 1.51 / 2 (75.7%):   2%|▏         | 2/100 [01:41<1:23:30, 51.13s/it]

...

,question,example_response,gold_doc_ids,reasoning,pred_response,SemanticF1
0,does using == in javascript ever make sense?,"Yes, using `==` in JavaScript can make sense and is convenient in ...","[5778, 5791, 5818]",The use of `==` in JavaScript can be misleading due to type coerci...,"Yes, using `==` in JavaScript can make sense in certain situations...",✔️ [0.667]
1,what is the difference between a virus and trojan?,The terms have a great deal of overlap and aren't necessarily mutu...,"[3768, 3769, 3888, 3890, 4046]","A virus and a Trojan are both types of malware, but they differ in...",A virus and a Trojan are different types of malware. A virus is a ...,✔️ [0.774]


🏃 View run eval at: http://localhost:5500/#/experiments/344816129373506955/runs/b07c1bab32b4464e83ab35e35a59514f
🧪 View experiment at: http://localhost:5500/#/experiments/344816129373506955


59.31

[Trace(request_id=6ce87b31a5a9490787082a0c72466ff4), Trace(request_id=2ff861482efc49f5a0e7e461c665da97), Trace(request_id=601b669e00114e56a7caf467410f5ebc), Trace(request_id=23616567361a4aae83b56df1642fec61), Trace(request_id=8c1fe220a33949b5bfe961ea34674fc9), Trace(request_id=18d003a934a64cd8971c4d7f098d9848), Trace(request_id=41ab560ff8f64bf79828013db77ebef1), Trace(request_id=b0393c437fc3440f9ff9c06967301b7d), Trace(request_id=c2c4c2e0e10f40f7a49621561603c3fd), Trace(request_id=104e3261472a40df86a7df3743b1b805)]

INFO:httpx:HTTP Request: POST http://localhost:8080/v1/chat/completions "HTTP/1.1 200 OK"
15:35:51 - LiteLLM:INFO: utils.py:1213 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
15:35:51 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
15:35:51 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/chat/completions "HTTP/1.1 200 OK"
15:35:52 - LiteLLM:INFO: utils.py:1213 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
15:35:52 - LiteLLM:INFO: cost_calculat

In [22]:
optimized_rag.save("optimized_rag.json")

loaded_rag = RAG()
loaded_rag.load("optimized_rag.json")

loaded_rag(question="cmd+tab does not work on hidden or minimized windows")

Prediction(
    reasoning='The issue with cmd+tab not working on hidden or minimized windows is due to a change in the behavior of the Mission Control system preference. The default setting now hides tabs when switching to an application, making it difficult to access minimized windows using cmd+tab. Various workarounds have been suggested, including using Control-CMD-F for full screen mode, switching to all open apps before hiding, or using third-party applications like iCanHazShortcut or Automator Services.',
    response='To access minimized windows using cmd+tab, try the following workarounds:\n- Use Control-CMD-F for full screen mode to access minimized windows.\n- Switch to all open apps before hiding.\n- Use the following AppleScript to switch to the first minimized window: `delay 0.5 set i to 0 tell application System Events set first_app to name of the first process whose frontmost is true repeat with p in every process if visible of p then set i to i + 1 end if end repeat rep

Trace(request_id=b7a0a8d5c5054c2b94793d42929fb6ce)

15:35:51 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
15:35:51 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: models/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: models/Llama-3.2-3B-Instruct-Q8_0.gguf
15:35:51 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf
15:35:51 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: models/Llama-3.2-3B-Instruct-Q8_0.gguf
INFO:LiteLLM:selected model name for cost calculation: models/Llama-3.2-3B-Instruct-Q8_0.gguf
15:35:57 - LiteLLM:INFO: cost_calculator.py:655 - selected model name fo